In [ ]:
total_lr = []
for num_subject in range(args.num_subjects):
    
    if num_subject in [0, 3, 28, 35, 49]: continue
    
    args['target_subject'] = num_subject
    args['target_subject'] = 0
    
    dataset = DroneDataset(base_path=args.BASE_PATH,
                            target_subject=args.target_subject,
                            task=args.task,
                            drop_channels=args.drop_channels, 
                            is_subject_independent=True)
    
    kfold = KFold(n_splits=args.k_folds, shuffle=True, random_state=args.SEED)
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        
        ### Set dataloader ###
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
        val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)
        
        train_dataloader = DataLoader(dataset,
                                        batch_size=args.batch_size,
                                        pin_memory=False,
                                        num_workers=args.num_workers,
                                        sampler=train_subsampler)
        val_dataloader = DataLoader(dataset,
                                    batch_size=args.batch_size,
                                    pin_memory=False,
                                    num_workers=args.num_workers,
                                    sampler=val_subsampler)
        
        model = LitModel(args.lr, args)
        early_stop_callback = EarlyStopping(monitor='val_acc',
                                            patience=args.early_stop,
                                            mode='max')
        
        sample = next(iter(train_dataloader))
        sample['data'] = sample['data'].type(torch.float)
        model(sample['data'])
        
        trainer = Trainer(
            progress_bar_refresh_rate=20,
            max_epochs=args.EPOCHS,
            gpus=int(args.GPU_NUM),
            callbacks=[early_stop_callback],
            auto_lr_find=True
        )

        lr_finder = trainer.tuner.lr_find(model, train_dataloader=train_dataloader, val_dataloaders=val_dataloader)
        lr_finder.results
        
#         fig = lr_finder.plot(suggest=True)
#         fig.show()
        
        print(f'LOG >>> Opimized lr: {lr_finder.suggestion()}')
        total_lr.append(lr_finder.suggestion())
    break
print(np.array(total_lr).mean())